## Spotify Listening Data Analysis

Author: Pavel Kukushkin

Dataset: export of listening data from 3 Spotify accounts, combined in a .csv file through various data transformations

In [1]:
import pandas as pd

In [8]:
spotify = pd.read_csv('https://raw.githubusercontent.com/pvlsson/spotify-history/master/data/pvl_spotify_history.csv', parse_dates=['listen_date'])

In [9]:
spotify.head()

,id,listen_date,username_hash,platform_group,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,shuffle,skipped,offline,incognito_mode,played_time,track_name
0,1,2019-03-30,b679e03a011213b4dc3fc8db84640a61,Windows,DE,Union,MRD,Sublimation,spotify:track:0FTAtFfdWexQwDDUIV9oRM,clickrow,trackdone,0,0,0,0,00:10:14,MRD - Union
1,2,2019-03-30,b679e03a011213b4dc3fc8db84640a61,Windows,DE,Ball Lightning,The Planetary Defence Side Project,Ball Lightning,spotify:track:0ENjvpFC7Qsn76wLQ3IKTf,trackdone,trackdone,0,0,0,0,00:08:27,The Planetary Defence Side Project - Ball Ligh...
2,3,2019-03-30,b679e03a011213b4dc3fc8db84640a61,Windows,DE,The Party We Could Have,Nathan Micay,The Party We Could Have,spotify:track:4Ub6kLgT87kBzpSj0Jgd8C,trackdone,trackdone,0,0,0,0,00:07:08,Nathan Micay - The Party We Could Have
3,4,2019-03-30,b679e03a011213b4dc3fc8db84640a61,Windows,DE,No Real Perspective,FNP,No Real Perspective,spotify:track:0UR4FNAXarQDN3AiAnA7lY,trackdone,trackdone,0,0,0,0,00:06:38,FNP - No Real Perspective
4,5,2019-03-31,b679e03a011213b4dc3fc8db84640a61,Windows,DE,Chickenwings - Original mix,Maureille,Chicken Tribulations,spotify:track:1XqBnLUpOSuZimbZHhoy02,trackdone,logout,0,0,0,0,00:05:02,Maureille - Chickenwings - Original mix


In [11]:
spotify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31414 entries, 0 to 31413
Data columns (total 17 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   id                                 31414 non-null  int64         
 1   listen_date                        31414 non-null  datetime64[ns]
 2   username_hash                      31414 non-null  object        
 3   platform_group                     31414 non-null  object        
 4   conn_country                       31414 non-null  object        
 5   master_metadata_track_name         27526 non-null  object        
 6   master_metadata_album_artist_name  27526 non-null  object        
 7   master_metadata_album_album_name   27526 non-null  object        
 8   spotify_track_uri                  27526 non-null  object        
 9   reason_start                       31414 non-null  object        
 10  reason_end                        

Looks like all data points were collected while listening to music online with incognito mode switched off.

In [10]:
spotify.describe()

,id,shuffle,skipped,offline,incognito_mode
count,31414.000000,31414.0,31414.000000,31414.0,31414.0
mean,15707.500000,0.0,0.059846,0.0,0.0
std,9068.585015,0.0,0.237205,0.0,0.0
min,1.000000,0.0,0.000000,0.0,0.0
25%,7854.250000,0.0,0.000000,0.0,0.0
50%,15707.500000,0.0,0.000000,0.0,0.0
75%,23560.750000,0.0,0.000000,0.0,0.0
max,31414.000000,0.0,1.000000,0.0,0.0
